## Dataset

In [19]:
import numpy as np
import pandas as pd
import timeit

In [5]:
df_bandung = pd.read_csv("/content/bandung.csv")
df_bengkulu = pd.read_csv("/content/bengkulu.csv")

In [6]:
coordinates_bengkulu = df_bengkulu[["lat", "long"]].values
coordinates_bandung = df_bandung[["lat", "long"]].values

start_latitude_bengkulu = -3.8606521
start_longitude_bengkulu = 102.3368196

start_latitude_bandung = -6.9146413
start_longitude_bandung = 107.5998627

# Models

## 1. TSP + Nearest Neighbors

In [11]:
import numpy as np

class TSP_NN():
    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate Haversine distance between two geographical coordinates.

        Parameters:
        - lat1 (float): Latitude of the first point.
        - lon1 (float): Longitude of the first point.
        - lat2 (float): Latitude of the second point.
        - lon2 (float): Longitude of the second point.

        Returns:
        - float: Haversine distance between the points.
        """
        R = 6371
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        distance = R * c
        return distance

    def nearest_neighbor(self, points, start_lat, start_lon):
        """
        Find nearest neighbors using the Nearest Neighbor algorithm.

        Parameters:
        - points (array): Array of coordinates.
        - start_lat (float): Latitude of the starting point.
        - start_lon (float): Longitude of the starting point.

        Returns:
        - list: Route containing indices of nearest neighbor points.
        - float: Total distance of the route.
        """
        self.route = []
        self.total_distance = None
        start_point = np.array([[start_lat, start_lon]])
        points = np.concatenate((points, start_point), axis=0)

        n = len(points)
        visited = np.zeros(n, dtype=bool)
        visited[-1] = True
        route = [n - 1]
        total_dist = 0

        for i in range(n - 1):
            last_point = route[-1]
            min_dist = float('inf')
            nearest_point = None
            for j in range(n):
                if not visited[j] and j != last_point:
                    dist = self.haversine_distance(points[last_point][0], points[last_point][1], points[j][0], points[j][1])
                    if dist < min_dist:
                        min_dist = dist
                        nearest_point = j
            visited[nearest_point] = True
            route.append(nearest_point)
            total_dist += min_dist

        route.remove(n - 1)
        total_dist += self.haversine_distance(points[route[-1]][0], points[route[-1]][1], points[route[0]][0], points[route[0]][1])

        return route, total_dist


## 2. TSP + Neural Network optimization

In [14]:
def model_v1():
  model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(2,)),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

def model_v2():
  model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(2,)),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1)
        ])
  model.compile(loss='mean_squared_error', optimizer='RMSProp')
  return model

In [15]:
import numpy as np
import tensorflow as tf

class TSP_DL():
    def __init__(self, embedding_size=32):
        self.embedding_size = embedding_size
        self.model = self.create_model()

    def create_model(self):
        model = model_v1()
        return model

    def train(self, X_train, y_train, epochs=50):
        self.model.fit(X_train, y_train, epochs=epochs)

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate Haversine distance between two geographical coordinates.

        Parameters:
        - lat1 (float): Latitude of the first point.
        - lon1 (float): Longitude of the first point.
        - lat2 (float): Latitude of the second point.
        - lon2 (float): Longitude of the second point.

        Returns:
        - float: Haversine distance between the points.
        """
        R = 6371
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        distance = R * c
        return distance

    def generate_sequences(self, points):
        sequences = []
        for i in range(len(points)):
            for j in range(len(points)):
                if i != j:
                    sequences.append(np.array([points[i], points[j]]))
        return np.array(sequences)

    def tsp_nn(self, coordinates, start_latitude, start_longitude):
        sequences = self.generate_sequences(coordinates)
        X_train = sequences[:, 0]
        y_train = np.array([np.where(np.all(sequence[1] == coordinates, axis=1))[0][0] for sequence in sequences])

        self.train(X_train, y_train)

        visited = [0]  # Kota awal
        current_city = 0  # Index kota awal
        start_point = np.array([start_latitude, start_longitude])

        while len(visited) < len(coordinates):
            distances = [self.haversine_distance(start_point[0], start_point[1],
                                                  coordinates[i][0], coordinates[i][1]) for i in range(len(coordinates))]
            next_city = np.argmax(self.model.predict(np.array([coordinates[current_city]])))
            while next_city in visited:
                distances[next_city] = 0  # Jangan pertimbangkan kembali kota yang sudah dikunjungi
                next_city = np.argmax(distances)
            visited.append(next_city)
            current_city = next_city

        total_distance = sum(self.haversine_distance(coordinates[visited[i]][0], coordinates[visited[i]][1],
                                                     coordinates[visited[i + 1]][0], coordinates[visited[i + 1]][1])
                            for i in range(len(coordinates) - 1))

        return visited, total_distance


## 3. TSP + Ant Colony

In [2]:
import numpy as np

class TSP_ACO():
    def __init__(self, num_ants=10, max_iter=100, alpha=1, beta=3, rho=0.1, Q=100):
        self.num_ants = num_ants  # Jumlah semut
        self.max_iter = max_iter  # Iterasi maksimum
        self.alpha = alpha  # Pengaruh pheromone
        self.beta = beta  # Pengaruh jarak
        self.rho = rho  # Tingkat evaporasi pheromone
        self.Q = Q  # Total pheromone yang dilepaskan
        self.route = None  # Jalur terbaik
        self.total_distance = None  # Jarak total terbaik

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate Haversine distance between two geographical coordinates.

        Parameters:
        - lat1 (float): Latitude of the first point.
        - lon1 (float): Longitude of the first point.
        - lat2 (float): Latitude of the second point.
        - lon2 (float): Longitude of the second point.

        Returns:
        - float: Haversine distance between the points.
        """
        R = 6371
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        distance = R * c
        return distance

    def ant_colony_optimization(self, points, start_lat, start_lon):
        start_point = np.array([[start_lat, start_lon]])
        points = np.concatenate((points, start_point), axis=0)

        n = len(points)
        distances = np.zeros((n, n))  # Matriks jarak antar titik
        pheromone = np.ones((n, n))  # Matriks pheromone
        np.fill_diagonal(distances, np.inf)  # Mengisi diagonal dengan infinitas

        # Menghitung jarak antar titik
        for i in range(n):
            for j in range(n):
                distances[i][j] = self.haversine_distance(points[i][0], points[i][1], points[j][0], points[j][1])

        # Algoritma ACO
        for _ in range(self.max_iter):
            ant_routes = []
            ant_distances = []

            for ant in range(self.num_ants):
                current_point = n - 1
                route = [current_point]
                total_dist = 0

                unvisited = list(range(n))
                unvisited.remove(current_point)

                while unvisited:
                    probabilities = (pheromone[current_point, unvisited] ** self.alpha) * ((1 / distances[current_point, unvisited]) ** self.beta)
                    probabilities /= probabilities.sum()
                    next_point = np.random.choice(unvisited, p=probabilities)

                    route.append(next_point)
                    total_dist += distances[current_point][next_point]

                    unvisited.remove(next_point)
                    current_point = next_point

                total_dist += distances[route[-1]][n - 1]  # Kembali ke titik awal
                ant_routes.append(route)
                ant_distances.append(total_dist)

            # Memperbarui pheromone
            delta_pheromone = np.zeros((n, n))
            for idx, route in enumerate(ant_routes):
                for i in range(n - 1):
                    delta_pheromone[route[i]][route[i + 1]] += self.Q / ant_distances[idx]
                delta_pheromone[route[-1]][n - 1] += self.Q / ant_distances[idx]

            pheromone = (1 - self.rho) * pheromone + delta_pheromone

        best_idx = np.argmin(ant_distances)
        self.route = ant_routes[best_idx][:-1]  # Hapus titik kembali ke awal dari jalur terbaik
        self.total_distance = ant_distances[best_idx] - distances[self.route[-1]][n - 1]  # Kurangi jarak kembali ke awal

        return self.route, self.total_distance


## 4. TSP + Bee Algorithms

In [16]:
import numpy as np

class TSP_BeeAlgorithm():
    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate Haversine distance between two geographical coordinates.

        Parameters:
        - lat1 (float): Latitude of the first point.
        - lon1 (float): Longitude of the first point.
        - lat2 (float): Latitude of the second point.
        - lon2 (float): Longitude of the second point.

        Returns:
        - float: Haversine distance between the points.
        """
        R = 6371
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        distance = R * c
        return distance

    def generate_random_solution(self, points):
        # Fungsi untuk menghasilkan solusi acak dari himpunan titik
        n = len(points)
        route = np.random.permutation(n)
        return route.tolist(), self.calculate_total_distance(route, points)

    def calculate_total_distance(self, route, points):
        # Fungsi untuk menghitung total jarak dari rute yang diberikan
        total_dist = 0
        for i in range(len(route) - 1):
            total_dist += self.haversine_distance(points[route[i]][0], points[route[i]][1],
                                                  points[route[i + 1]][0], points[route[i + 1]][1])
        total_dist += self.haversine_distance(points[route[-1]][0], points[route[-1]][1],
                                              points[route[0]][0], points[route[0]][1])
        return total_dist

    def run_bee_algorithm(self, points, start_lat, start_lon, num_iterations, num_bees):
        # Menggabungkan titik-titik dengan titik awal
        start_point = np.array([[start_lat, start_lon]])
        points = np.concatenate((points, start_point), axis=0)

        best_route, best_distance = self.generate_random_solution(points)

        for _ in range(num_iterations):
            for _ in range(num_bees):
                current_route, current_distance = self.generate_random_solution(points)
                if current_distance < best_distance:
                    best_route = current_route
                    best_distance = current_distance

        best_route.remove(len(points) - 1)  # Hapus titik awal dari rute terbaik
        return best_route, best_distance

## 5. TSP + Graph Neural Network

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model

class GraphNeuralNetwork(Model):
    def __init__(self, hidden_dim):
        super(GraphNeuralNetwork, self).__init__()
        self.hidden_dim = hidden_dim
        self.dense1 = layers.Dense(hidden_dim, activation='relu')
        self.dense2 = layers.Dense(hidden_dim, activation='relu')

    def call(self, inputs):
        x, adj_matrix = inputs
        x = tf.matmul(adj_matrix, x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

class TSP_GNN():
    def __init__(self, hidden_dim):
        self.model = GraphNeuralNetwork(hidden_dim)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
        self.loss_fn = tf.keras.losses.MeanSquaredError()

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate Haversine distance between two geographical coordinates.

        Parameters:
        - lat1 (float): Latitude of the first point.
        - lon1 (float): Longitude of the first point.
        - lat2 (float): Latitude of the second point.
        - lon2 (float): Longitude of the second point.

        Returns:
        - float: Haversine distance between the points.
        """
        R = 6371
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        distance = R * c
        return distance


    def create_adjacency_matrix(self, points):
        n = len(points)
        adjacency_matrix = np.zeros((n, n))

        for i in range(n):
            for j in range(n):
                if i != j:
                    distance = self.haversine_distance(points[i][0], points[i][1], points[j][0], points[j][1])
                    adjacency_matrix[i][j] = 1.0 / distance  # Invers dari jarak sebagai bobot

        return adjacency_matrix.astype(np.float32)

    def train(self, points, start_lat, start_lon, epochs=100):
        start_point = np.array([[start_lat, start_lon]])
        points = np.concatenate((points, start_point), axis=0)

        adjacency_matrix = self.create_adjacency_matrix(points)
        adjacency_matrix /= adjacency_matrix.max()

        input_dim = len(points[0])
        hidden_dim = 64

        # Persiapan data untuk pelatihan (dalam kasus ini, menggunakan contoh sederhana yaitu titik-titik)
        x = points.astype(np.float32)
        x = tf.constant(x)
        adjacency_matrix = tf.constant(adjacency_matrix)

        # Pelatihan GNN
        for epoch in range(epochs):
            with tf.GradientTape() as tape:
                outputs = self.model((x, adjacency_matrix))
                loss_value = self.loss_fn(outputs, x)

            gradients = tape.gradient(loss_value, self.model.trainable_weights)
            self.optimizer.apply_gradients(zip(gradients, self.model.trainable_weights))

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss_value.numpy()}")

    def infer(self, points, start_lat, start_lon):
        start_point = np.array([[start_lat, start_lon]])
        points = np.concatenate((points, start_point), axis=0)

        adjacency_matrix = self.create_adjacency_matrix(points)
        adjacency_matrix /= adjacency_matrix.max()

        x = points.astype(np.float32)
        x = tf.constant(x)
        adjacency_matrix = tf.constant(adjacency_matrix)

        outputs = self.model((x, adjacency_matrix))

        route_indices = tf.argsort(outputs, axis=1)[:, :-1]  # Dapatkan urutan indeks terurut dari output GNN

        # Dapatkan indeks rute terpendek dari hasil inferensi
        shortest_route_index = tf.argmin(tf.reduce_sum(outputs, axis=1))

        # Ambil rute terpendek berdasarkan indeks
        route = route_indices[shortest_route_index].numpy()

        # Hitung total jarak untuk rute yang dihasilkan
        total_distance = 0
        for i in range(len(route) - 1):
            total_distance += self.haversine_distance(
                points[route[i]][0], points[route[i]][1],
                points[route[i + 1]][0], points[route[i + 1]][1]
            )
        total_distance += self.haversine_distance(
            points[route[-1]][0], points[route[-1]][1],
            points[route[0]][0], points[route[0]][1]
        )

        return route, total_distance


# Comparison

### TSP + Nearest Neighbors

In [12]:
tsp = TSP_NN()

start_bandung = timeit.default_timer()
route_bandung, total_distance_bandung = tsp.nearest_neighbor(coordinates_bandung, start_latitude_bandung, start_longitude_bandung)
print("Best route:", route_bandung)
print("Best distance:", total_distance_bandung, "kilometer")
stop_bandung = timeit.default_timer()
execution_time = stop_bandung - start_bandung
print(execution_time, "s")

start_bengkulu = timeit.default_timer()
route_bengkulu, total_distance_bengkulu = tsp.nearest_neighbor(coordinates_bengkulu, start_latitude_bengkulu, start_longitude_bengkulu)
print("Best route:", route_bengkulu)
print("Best distance:", total_distance_bengkulu, "kilometer")
stop_bengkulu = timeit.default_timer()
execution_time = stop_bengkulu - start_bengkulu
print(execution_time, "s")

Best route: [73, 92, 11, 119, 16, 12, 75, 1, 60, 53, 82, 25, 74, 29, 54, 123, 65, 83, 72, 77, 6, 43, 63, 48, 38, 10, 40, 49, 47, 2, 91, 32, 36, 3, 68, 69, 46, 17, 66, 56, 37, 44, 67, 79, 23, 20, 5, 98, 120, 121, 116, 122, 34, 28, 99, 64, 35, 78, 50, 9, 62, 106, 81, 115, 18, 61, 31, 101, 4, 42, 51, 24, 30, 13, 103, 90, 57, 15, 102, 8, 59, 21, 100, 104, 55, 86, 114, 52, 71, 45, 58, 27, 84, 41, 85, 26, 80, 70, 33, 39, 112, 113, 109, 7, 108, 107, 110, 111, 94, 95, 93, 96, 22, 87, 118, 76, 19, 117, 88, 0, 14, 97, 89, 105]
Best distance: 430.48120969787584 kilometer
0.12257403400002431 s
Best route: [21, 39, 1, 5, 33, 18, 6, 0, 7, 46, 30, 37, 23, 16, 36, 11, 38, 8, 34, 4, 2, 13, 3, 35, 43, 10, 14, 45, 26, 12, 28, 25, 44, 40, 22, 27, 20, 47, 19, 9, 31, 41, 42, 32, 29, 24, 17, 15]
Best distance: 693.7076668013398 kilometer
0.02009811000004902 s


### TSP + Neural Network

In [18]:
dl = TSP_DL()

start_bandung = timeit.default_timer()
route_bandung, total_distance_bandung = dl.tsp_nn(coordinates_bandung, start_latitude_bandung, start_longitude_bandung)
print("Best route:", route_bandung)
print("Best distance:", total_distance_bandung, "kilometer")
stop_bandung = timeit.default_timer()
execution_time = stop_bandung - start_bandung
print(execution_time, "s")

start_bengkulu = timeit.default_timer()
route_bengkulu, total_distance_bengkulu = dl.tsp_nn(coordinates_bengkulu, start_latitude_bengkulu, start_longitude_bengkulu)
print("Best route:", route_bengkulu)
print("Best distance:", total_distance_bengkulu, "kilometer")
stop_bengkulu = timeit.default_timer()
execution_time = stop_bengkulu - start_bengkulu
print(execution_time, "s")

Epoch 1/50
477/477 [==============================] - 3s 4ms/step - loss: 1316.8210
Epoch 2/50
477/477 [==============================] - 1s 3ms/step - loss: 1292.1444
Epoch 3/50
477/477 [==============================] - 1s 2ms/step - loss: 1293.7610
Epoch 4/50
477/477 [==============================] - 1s 3ms/step - loss: 1291.2545
Epoch 5/50
477/477 [==============================] - 1s 3ms/step - loss: 1287.3231
Epoch 6/50
477/477 [==============================] - 1s 2ms/step - loss: 1289.1897
Epoch 7/50
477/477 [==============================] - 1s 3ms/step - loss: 1289.8741
Epoch 8/50
477/477 [==============================] - 1s 2ms/step - loss: 1288.9912
Epoch 9/50
477/477 [==============================] - 1s 2ms/step - loss: 1288.1312
Epoch 10/50
477/477 [==============================] - 1s 3ms/step - loss: 1293.2501
Epoch 11/50
477/477 [==============================] - 2s 4ms/step - loss: 1287.3572
Epoch 12/50
477/477 [==============================] - 2s 4ms/step - loss:

### TSP + Ant Colony

In [20]:
aco = TSP_ACO()

start_bandung = timeit.default_timer()
route_bandung, total_distance_bandung = aco.ant_colony_optimization(coordinates_bandung, start_latitude_bandung, start_longitude_bandung)
print("Best route:", route_bandung)
print("Best distance:", total_distance_bandung, "kilometer")
stop_bandung = timeit.default_timer()
execution_time = stop_bandung - start_bandung
print(execution_time, "s")

start_bengkulu = timeit.default_timer()
route_bengkulu, total_distance_bengkulu = aco.ant_colony_optimization(coordinates_bengkulu, start_latitude_bengkulu, start_longitude_bengkulu)
print("Best route:", route_bengkulu)
print("Best distance:", total_distance_bengkulu, "kilometer")
stop_bengkulu = timeit.default_timer()
execution_time = stop_bengkulu - start_bengkulu
print(execution_time, "s")

Best route: [124, 73, 92, 54, 53, 60, 82, 25, 1, 74, 29, 2, 47, 49, 40, 10, 91, 48, 38, 77, 6, 43, 63, 61, 31, 18, 79, 67, 44, 37, 115, 23, 20, 5, 98, 120, 121, 116, 122, 102, 90, 103, 57, 15, 30, 13, 4, 101, 51, 42, 24, 86, 55, 59, 21, 8, 100, 104, 27, 58, 45, 71, 52, 114, 36, 32, 3, 68, 69, 46, 17, 66, 75, 119, 16, 12, 11, 72, 83, 65, 123, 84, 41, 85, 26, 80, 33, 70, 39, 112, 113, 109, 7, 108, 107, 110, 111, 95, 93, 96, 22, 94, 105, 89, 19, 117, 76, 118, 87, 56, 97, 14, 0, 50, 35, 78, 9, 62, 106, 99, 28, 64, 34, 81]
Best distance: 396.56890560601954 kilometer
10.99839375700003 s
Best route: [48, 21, 39, 1, 5, 33, 18, 6, 0, 7, 46, 30, 37, 23, 16, 36, 11, 38, 8, 34, 4, 2, 13, 3, 35, 43, 10, 14, 45, 26, 12, 28, 25, 42, 32, 41, 31, 9, 19, 47, 22, 27, 20, 40, 44, 29, 17, 24]
Best distance: 545.6667916132978 kilometer
2.7803475470000194 s


### TSP + Bee Algorithms

In [23]:
bee = TSP_BeeAlgorithm()

start_bandung = timeit.default_timer()
route_bandung, total_distance_bandung = bee.run_bee_algorithm(coordinates_bandung, start_latitude_bandung, start_longitude_bandung, num_iterations=100, num_bees=len(coordinates_bandung))
print("Best route:", route_bandung)
print("Best distance:", total_distance_bandung, "kilometer")
stop_bandung = timeit.default_timer()
execution_time = stop_bandung - start_bandung
print(execution_time, "s")

start_bengkulu = timeit.default_timer()
route_bengkulu, total_distance_bengkulu = bee.run_bee_algorithm(coordinates_bengkulu, start_latitude_bengkulu, start_longitude_bengkulu, num_iterations=100, num_bees=len(coordinates_bengkulu))
print("Best route:", route_bengkulu)
print("Best distance:", total_distance_bengkulu, "kilometer")
stop_bengkulu = timeit.default_timer()
execution_time = stop_bengkulu - start_bengkulu
print(execution_time, "s")

Best route: [56, 71, 10, 53, 16, 68, 50, 76, 46, 95, 101, 12, 116, 80, 112, 47, 31, 28, 60, 114, 34, 37, 120, 62, 90, 51, 22, 54, 43, 30, 79, 78, 99, 97, 35, 61, 42, 92, 17, 52, 9, 7, 107, 113, 11, 13, 67, 85, 117, 105, 109, 111, 72, 64, 57, 27, 0, 118, 44, 41, 4, 23, 103, 59, 55, 36, 102, 38, 100, 40, 70, 82, 110, 108, 94, 75, 19, 119, 66, 3, 65, 2, 58, 106, 15, 122, 32, 121, 69, 39, 98, 73, 26, 77, 86, 33, 5, 87, 63, 88, 89, 25, 91, 81, 1, 104, 20, 49, 24, 14, 21, 48, 45, 83, 96, 6, 84, 74, 93, 29, 18, 115, 8, 123]
Best distance: 1686.4762777487238 kilometer
35.92384777899997 s
Best route: [5, 20, 43, 45, 8, 27, 18, 37, 0, 23, 33, 21, 38, 6, 31, 26, 36, 19, 12, 14, 44, 2, 9, 28, 16, 46, 40, 22, 13, 7, 25, 47, 34, 4, 30, 11, 3, 10, 32, 42, 1, 15, 17, 24, 29, 41, 35, 39]
Best distance: 1449.2009202249963 kilometer
5.691782644 s


### TSP + GNN

In [29]:
gnn_bandung = TSP_GNN(len(coordinates_bandung))
gnn_bengkulu = TSP_GNN(len(coordinates_bengkulu))

start_bandung = timeit.default_timer()
best_route_bandung, best_distance_bandung = gnn_bandung.infer(coordinates_bandung, start_latitude_bandung, start_longitude_bandung)
print("Best route:", best_route_bandung)
print("Best distance:", best_distance_bandung, "kilometer")
stop_bandung = timeit.default_timer()
execution_time = stop_bandung - start_bandung
print(execution_time, "s")

start_bengkulu = timeit.default_timer()
best_route_bengkulu, best_distance_bengkulu = gnn_bengkulu.infer(coordinates_bengkulu, start_latitude_bengkulu, start_longitude_bengkulu)
print("Best route:", best_route_bengkulu)
print("Best distance:", best_distance_bengkulu, "kilometer")
stop_bengkulu = timeit.default_timer()
execution_time = stop_bengkulu - start_bengkulu
print(execution_time, "s")

Best route: [  2   4   7  11  12  14  15  22  23  27  28  29  30  31  37  39  41  43
  45  46  47  50  51  53  54  56  58  60  63  66  67  68  73  74  75  76
  77  78  79  82  83  84  86  92  93  94  95  97 100 102 103 104 106 107
 108 111 113 117 119  17 123  52   8  34 118  69 109  19  44 121  42  98
  89  32  65  16  57 101 114  85  91  55  70  59  38  48 115 122  80  33
  90  25  13  35  62  81 112 120  20   0 105  18  21  72  24  49  36  40
  87  99   3   5  61  64   6 116   9 110  71   1  88  10  96]
Best distance: 1741.2453305959968 kilometer
0.24231646199996248 s
Best route: [ 0  1  4  5  6 10 12 14 16 17 18 19 20 21 23 24 26 28 30 31 34 35 36 37
 38 43 46  2  7 22 11 47 27 44 39  8 41  3 33 13 45 25 40 32  9 15 29]
Best distance: 1807.8571190915716 kilometer
0.04945280299989463 s


# Report


| Algorithm                    | Location | Best Distance (km) | Execution Time (s) |
|------------------------------|----------|--------------------|--------------------|
| TSP + Nearest Neighbors      | Bandung  | 430.481            | 0.123              |
| TSP + Nearest Neighbors      | Bengkulu | 693.708            | 0.020              |
| TSP + Deep Neural Network    | Bandung  | 1113.591           | 162.740            |
| TSP + Deep Neural Network    | Bengkulu | 852.011            | 25.076             |
| TSP + ACO                    | Bandung  | 396.569            | 10.998             |
| TSP + ACO                    | Bengkulu | 545.667            | 2.780              |
| TSP + Bee Algorithms         | Bandung  | 1686.476           | 35.924             |
| TSP + Bee Algorithms         | Bengkulu | 1449.201           | 5.692              |
| TSP + Graph Neural Network   | Bandung  | 1741.245           | 0.242              |
| TSP + Graph Neural Network   | Bengkulu | 1807.857            | 0.049              |
